# Welcome to this text_to_text_end_to_end generator framework
about us

This notebook help you to do massive translation from up to 200 languages .


Autor : Mowoum Foupouagnigni , UMMISCO intern ENSPY 2025 , computer engineering
email : foupouagnignimowoum@gmail.com

#Part 1 : Dependences installation


for dependance issue with this notebook , please check the requirement.txt file provided .

In [1]:
!pip install transformers torch sentencepiece # Install necessary libraries

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

#Part 2 : Initialization

## Model initialisation

In [2]:
from transformers import pipeline
import torch

# 1. Load the NLLB-200 model for translation
# Check for GPU availability
device = 0 if torch.cuda.is_available() else -1 # 0 for GPU, -1 for CPU

print(f"Loading translation model to device: {'GPU' if device == 0 else 'CPU'}")
translator = pipeline(
    "translation",
    model="facebook/nllb-200-distilled-600M",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32, # Use float16 for faster inference on GPU
    device=device
)
print("Model loaded successfully!")

Loading translation model to device: GPU


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


Model loaded successfully!


## code Initialization

In [3]:
import os

def translate(data_source_dir_path, source_language, target_language, data_output_dir_path, translator):
    for fichier in os.listdir(data_source_dir_path):
        chemin_fichier = os.path.join(data_source_dir_path, fichier)

        if chemin_fichier.endswith(".txt"):
            # Lire le fichier d'entrée
            with open(chemin_fichier, "r", encoding="utf-8") as f:
                sentences = f.readlines()

            # Nettoyer les phrases (supprimer les espaces)
            sentences = [s.strip() for s in sentences if s.strip()]

            if not sentences:
                print(f"Warning: No valid sentences found in {chemin_fichier}. Output file will be empty.")
            else:
                print(f"Translating {len(sentences)} sentences from {fichier}...")

                # Traduire les phrases
                translated_results = translator(
                    sentences,
                    src_lang=source_language,
                    tgt_lang=target_language,
                )

                # Extraire le texte traduit
                translated_sentences = [result["translation_text"] for result in translated_results]

                # Écrire les phrases traduites dans le dossier de sortie
                output_file_name = os.path.join(data_output_dir_path, os.path.basename(fichier))

                with open(output_file_name, "w", encoding="utf-8") as f:
                    for sentence in translated_sentences:
                        f.write(sentence + "\n")

                print(f"Translation complete! Translated text saved to: {output_file_name}")


## Part 3 : Storage initialization

In [5]:
#Here we used google drive cloud as storage cloud
from google.colab import drive
# 1. Mount your google drive cloud storage
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#Part 3 : translation

To see the correct language code , please check the NLLB200 paper present in the documentations folder page 13 .
exemple
english code :  eng_Latn
french code  :  fra_Latn

In [ ]:
if __name__ == "__main__":

  print("exemple english code :  eng_Latn || french code  :  fra_Latn")
  source_language=input("please enter de source language code: ")
  target_language=input("please enter the target language code: ")

  principal_folder_path = input("please enter the path of the /translator_text_files folder given to you , and we will do the rest \n")
  data_source_dir=os.path.join(principal_folder_path,"data")
  data_output_dir=os.path.join(principal_folder_path,"generated_data")


translate(data_source_dir,source_language,target_language,data_output_dir,translator)

print("translation finished")


exemple english code :  eng_Latn || french code  :  fra_Latn
Translating 1 sentences from phrase_0001.txt...
Translation complete! Translated text saved to: /content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/Framework_final/verified/translator_text_files/generated_data/phrase_0001.txt
Translating 1 sentences from phrase_0002.txt...
Translation complete! Translated text saved to: /content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/Framework_final/verified/translator_text_files/generated_data/phrase_0002.txt
Translating 1 sentences from phrase_0003.txt...
Translation complete! Translated text saved to: /content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/Framework_final/verified/translator_text_files/generated_data/phrase_0003.txt
Translating 1 sentences from phrase_0004.txt...
Translation complete! Translated text saved to: /content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/Framework_final/verified/translator_text_files/generated_data

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Streaming output truncated to the last 5000 lines.
Translation complete! Translated text saved to: /content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/Framework_final/verified/translator_text_files/generated_data/phrase_2034.txt
Translating 1 sentences from phrase_2035.txt...
Translation complete! Translated text saved to: /content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/Framework_final/verified/translator_text_files/generated_data/phrase_2035.txt
Translating 1 sentences from phrase_2036.txt...
Translation complete! Translated text saved to: /content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/Framework_final/verified/translator_text_files/generated_data/phrase_2036.txt
Translating 1 sentences from phrase_2037.txt...
Translation complete! Translated text saved to: /content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/Framework_final/verified/translator_text_files/generated_data/phrase_2037.txt
Translating 1 sentences from phrase_2038.

# Génération du Fichier requierement.txt et sauvegarde dans la racine du framework

In [ ]:
#Génération du fichier de versionning des dépendances
requirements_path = os.path.join(principal_folder_path, 'requirements.txt')
!pip freeze > {requirements_path}

print("Fin de la sauvegarde du fichier des dépendances global du notebook , go and check : "+ principal_folder_path)

In [9]:
import os

def split_sentences_into_files(input_filepath, output_directory="phrases_separees"):
    """
    Lit chaque phrase d'un fichier d'entrée et la sauvegarde dans des fichiers .txt séparés,
    numérotés séquentiellement. Gère les lignes vides ou nulles.

    Args:
        input_filepath (str): Le chemin vers le fichier texte contenant toutes les phrases.
        output_directory (str): Le répertoire où les fichiers de phrases seront sauvegardés.
                                 Sera créé s'il n'existe pas.
    """
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
        print(f"Dossier de sortie créé : '{output_directory}'")

    sentence_count = 3425
    try:
        with open(input_filepath, 'r', encoding='utf-8') as infile:
            for line in infile:
                # Nettoyer la ligne (supprimer les espaces blancs au début/fin, y compris les retours chariot)
                stripped_line = line.strip()

                # Ignorer les lignes vides ou qui ne contiennent que des espaces
                if not stripped_line:
                    continue

                sentence_count += 1
                # Formatage du nom de fichier avec un numéro séquentiel (ex: phrase_001.txt)
                output_filename = os.path.join(output_directory, f"phrase_{sentence_count:04d}.txt")

                with open(output_filename, 'w', encoding='utf-8') as outfile:
                    outfile.write(stripped_line)

                print(f"Phrase {sentence_count} sauvegardée dans '{output_filename}'")

        print(f"\nTraitement terminé. {sentence_count} phrases ont été sauvegardées.")

    except FileNotFoundError:
        print(f"Erreur : Le fichier d'entrée '{input_filepath}' est introuvable. Veuillez vérifier le chemin.")
    except Exception as e:
        print(f"Une erreur inattendue est survenue : {e}")


In [10]:

# --- Instructions d'utilisation ---
if __name__ == "__main__":
    # Remplacez 'votre_fichier_de_phrases.txt' par le chemin réel de votre fichier de phrases.
    # Assurez-vous que ce fichier existe dans le même répertoire que votre script Python
    # ou indiquez un chemin complet (ex: '/chemin/vers/mon/fichier/mes_phrases.txt').
    input_file = "/content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/datasets/autre dataset/Fr-En/unique_words.txt"

    # Vous pouvez changer le nom du dossier de sortie si vous le souhaitez.
    # Par défaut, les fichiers seront créés dans un dossier 'phrases_separees'
    # dans le même répertoire que votre script Python.
    output_folder = "/content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/datasets/autre dataset/Fr-En/datasetIncrémentalFr"

    split_sentences_into_files(input_file, output_folder)

Phrase 3426 sauvegardée dans '/content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/datasets/autre dataset/Fr-En/datasetIncrémentalFr/phrase_3426.txt'
Phrase 3427 sauvegardée dans '/content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/datasets/autre dataset/Fr-En/datasetIncrémentalFr/phrase_3427.txt'
Phrase 3428 sauvegardée dans '/content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/datasets/autre dataset/Fr-En/datasetIncrémentalFr/phrase_3428.txt'
Phrase 3429 sauvegardée dans '/content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/datasets/autre dataset/Fr-En/datasetIncrémentalFr/phrase_3429.txt'
Phrase 3430 sauvegardée dans '/content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/datasets/autre dataset/Fr-En/datasetIncrémentalFr/phrase_3430.txt'
Phrase 3431 sauvegardée dans '/content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/datasets/autre dataset/Fr-En/datasetIncrémentalFr/phrase_3431.txt'
Phrase 3432 sauvegardé

In [8]:
# Lire le fichier et traiter les mots
def process_words(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        # Lire le contenu du fichier et séparer par le backslash
        words = file.read().strip().split('\\')

    # Supprimer les doublons en utilisant un ensemble
    unique_words = set(words)

    # Trier les mots (facultatif)
    sorted_words = sorted(unique_words)

    # Écrire les mots uniques dans un nouveau fichier
    with open('/content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/datasets/autre dataset/Fr-En/unique_words.txt', 'w', encoding='utf-8') as output_file:
        for word in sorted_words:
            output_file.write(word + '\n')

# Remplacez 'votre_fichier.txt' par le chemin de votre fichier
process_words('/content/gdrive/MyDrive/UMMISCO_ENSPY_stage_mowoum_foupouagnigni/datasets/autre dataset/Fr-En/motsFr.txt')

#END THANKS YOU